In [ ]:
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
import sagemaker
import torch

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [ ]:
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
train_input_path = f"s3://{bucket}/imdb/data/small/train.csv"
test_input_path = f"s3://{bucket}/imdb/data/small/test.csv"

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")

In [ ]:
metric_definitions=[
    {'Name': 'traning:loss', 'Regex': "'loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:loss', 'Regex': "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:accuracy', 'Regex': "'eval_accuracy': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:f1', 'Regex': "'eval_f1': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:precision', 'Regex': "'eval_precision': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:recall', 'Regex': "'eval_recall': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:runtime', 'Regex': "'eval_runtime': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'validation:samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'learning_rate', 'Regex': "'learning_rate': ([0-9]+(.|e\-)[0-9]+),?"},
    {'Name': 'epoch', 'Regex': "'epoch': ([0-9]+(.|e\-)[0-9]+),?"}]

## Training

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters = {'epochs': 1,
                   'model_name': 'distilbert-base-uncased'
                   }

# create the Estimator
estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./source',
    instance_type='ml.g4dn.xlarge',  # Note: needs to be an instance with gpu
    instance_count=1,
    role=role,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    hyperparameters=hyperparameters
    metric_definitions=metric_definitions,
)

In [ ]:
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "train-batch-size": CategoricalParameter([32, 64]),
}

Next we'll specify the objective metric that we'd like to tune and its definition, which includes the regular expression (Regex) needed to extract that metric from the CloudWatch logs of the training job. In this particular case, our script emits average loss value and we will use it as the objective metric, we also set the objective_type to be 'minimize', so that hyperparameter tuning seeks to minize the objective metric when searching for the best hyperparameter setting. By default, objective_type is set to 'maximize'.

In [ ]:
objective_metric_name = "validation loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "validation loss", "Regex": "'eval_loss': ([0-9]+(.|e\-)[0-9]+),?"}]

Now, we'll create a `HyperparameterTuner` object, to which we pass:
- The PyTorch estimator we created above
- Our hyperparameter ranges
- Objective metric name and definition
- Tuning resource configurations such as Number of training jobs to run in total and how many training jobs can be run in parallel.

In [ ]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=9,
    max_parallel_jobs=3,
    objective_type=objective_type,
)